In [1]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.spatial.distance import cosine

In [2]:
import sys
sys.path.append('../')

In [3]:
from Text_summarizer.preprocessing import preprocess
from Text_summarizer.embedding import Embeddings

In [21]:
import numpy as np


class Summarize:
    def __init__(self, *args, **kwargs):
        """Initialize the class by loading the word embeddings."""
        print('Loading Embedding')
        self.word_vec = Embeddings().load_vector()
        self.key_to_index = self.word_vec.key_to_index
        self.vector_size = self.word_vec.vector_size
        print('Embedding is now loaded')

    def generate_centroid_tfidf(self, sentences):
        """Generate TF-IDF centroid of important words in the text."""
        tf = TfidfVectorizer()
        tfidf_matrix = tf.fit_transform(sentences).toarray()
        tfidf_sum = np.sum(tfidf_matrix, axis=0)
        tfidf_max = tfidf_sum.max()
        tfidf_normalized = tfidf_sum / tfidf_max

        words = tf.get_feature_names_out()
        important_terms = [
            word for i, word in enumerate(words) if word in self.word_vec.key_to_index and tfidf_normalized[i] >= 0.2
        ]
        centroid = np.mean([self.word_vec[word] for word in important_terms],
                           axis=0) if important_terms else np.zeros(self.vector_size)
        return centroid

    def sentence_vectorizer(self, sentences):
        """Convert each sentence into a vector by averaging word embeddings."""
        sentence_vectors = {}
        for i, sentence in enumerate(sentences):
            words = sentence.split()
            sentence_vec = np.zeros(self.vector_size)
            word_count = 0
            for word in words:
                if word in self.word_vec.key_to_index:
                    sentence_vec = np.add(sentence_vec, self.word_vec[word])
                    word_count += 1
            if word_count > 0:
                sentence_vec /= word_count  # Average the word vectors
            sentence_vectors[i] = sentence_vec
        return sentence_vectors

    def sentence_selection(self, centroid, sentence_vectors, summary_length):
        """Select the most important sentences based on similarity to the centroid."""
        similarities = []
        for sentence_id, sentence_vec in sentence_vectors.items():
            similarity = 1 - cosine(centroid, sentence_vec)
            similarities.append((sentence_id, similarity))

        # Sort by similarity (descending order) and select top sentences
        ranked_sentences = sorted(
            similarities, key=lambda x: x[1], reverse=True)

        selected_sentences = []
        total_length = 0
        for sentence_id, _ in ranked_sentences:
            selected_sentences.append(sentence_id)
            # Add sentence length to total
            total_length += len(sentence_vectors[sentence_id])
            if total_length >= summary_length:
                break

        return sorted(selected_sentences)

    def combine_sentence(self, selected_sentence_ids, sentences):
        """Combine the selected sentences into a final summary."""
        return '। '.join([sentences[i] for i in selected_sentence_ids])

    def show_summary(self, text, length_sentence_predict=0.01):
        """Main function to generate a summary from the input text."""
        sentences = preprocess(text)

        # Calculate the total length of the text in characters
        total_text_length = len(text)

        # Dynamically adjust the threshold based on the text length
        if total_text_length < 500:  # Short text
            length_sentence_predict = max(0.8, length_sentence_predict)
        elif total_text_length < 1500:  # Medium-length text
            length_sentence_predict = max(0.5, length_sentence_predict)
        else:  # Long text
            length_sentence_predict = max(0.2, length_sentence_predict)

        # Generate centroid of important terms
        centroid = self.generate_centroid_tfidf(sentences)
        sentence_vectors = self.sentence_vectorizer(sentences)

        # Determine summary length (either by character limit or number of sentences)
        if length_sentence_predict < 1:
            total_char_count = sum(len(sentence) for sentence in sentences)
            length_sentence_predict = int(
                total_char_count * length_sentence_predict)

        # Select important sentences based on centroid similarity
        selected_sentence_ids = self.sentence_selection(
            centroid, sentence_vectors, length_sentence_predict)
        return f"{self.combine_sentence(selected_sentence_ids, sentences)}।"

In [5]:
news = """
१ पुस, काठमाडौं । नापी विभागले लिम्पियाधुरालाई समेटेर नयाँ नक्सा तयार पार्न थालेको छ । सरकारको निर्देशनबमोजिम भारतबाट अतिक्रमित नेपाली भूभागलाई समेटेर नयाँ नक्सा तयार गर्ने प्रक्रिया अगाडि बढेको विभागले जनाएको छ । ‘नयाँ नक्सा तयार गर्ने गरी प्रक्रिया अगाडि बढेको छ’, विभागका सूचना अधिकारी दामोदर ढकालले अनलाइनखबरसँग भने । संघीय संसदका राज्य व्यवस्था समिति र अन्तर्राष्ट्रिय सम्बन्ध समितिले कालापानी, लिपुलेक, लिम्पियाधुरा र सुस्तालगायतका अतिक्रमित नेपाली भूभाग समेटेर नयाँ नक्सा सार्वजनिक गर्न सरकारलाई निर्देशन दिएका छन् । सन् १८१६ मा भएको सुगौली सन्धिअनुसार नेपालको पश्चिमी सिमाना काली (महाकाली) नदी हो । सन्धिअनुसार लिम्पियाधुराबाट बग्ने काली नदी पूर्वका सम्पूर्ण भूभाग नेपालको हो । ती सम्पूर्ण भूभाग समेटेर नक्सा सार्वजनिक गर्न दिएको निर्देशन पालना नभएको भनेर आलोचना भइरहेका बेला विभागले सरकारको निर्देशनमा नयाँ नक्सा बनाउन लागेको हो । विभागका सूचना अधिकारी ढकाल भन्छन्, ‘नापी विभाग सरकारको निर्देशन पालना गर्ने निकाय हो, निर्देशन आएको छ । बाँकी निर्णय उचित समय आएपछि हुन्छ ।’ यो अवस्थामा नक्सा जारी गर्नुभन्दा पहिले कूटनीतिक पहल गर्नुपर्ने विभागका अर्का एक अधिकारी बताउँछन् । ‘तर, यो विषयलाई अन्तर्राष्ट्रियकरण गर्नुपर्ने आवाजसमेत उठिरहेको भन्दै सरकारले प्रमाणका आधारमा जति नेपाली भूभाग हो त्यसलाई समेटेर नयाँ नक्सा तयार गर्न भनेको छ । हामी सरकारले निर्णय गरेका दिन, मागेका दिन नयाँ नक्सा उपलब्ध गराउन सक्ने गरी तयारी गर्दैछौं’ ती अधिकारीले भने । ‘नयाँ नक्सा जारी गर्ने अनुकुल वातावरण’ विज्ञहरुले पनि भारतले नयाँ नक्सा जारी गरेर सीमा बहसलाई तताएका बेला नेपालले पनि आफ्ना भूभागलाई समेटेर नयाँ नक्सा जारी गर्न सक्नुपर्ने बताएका छन् । त्यसका लागि अनुकुल वातावरण तयार भएको परराष्ट्रविद हिरण्यलाल श्रेष्ठ बताउँछन् । मिनी संसदको सक्रियताले अन्तर्राष्ट्रिय स्तरमा नेपाल सीमा समस्यामा चिन्तित छ भन्ने सकारात्मक सन्देश गइरहेको उनको भनाइ छ । श्रेष्ठका अनुसार सीमा समस्या समाधानमा सरकारले राष्ट्रिय सहमति कायम गर्नु, यो विषयमा सर्वोच्च अदालतमा पुग्नु र संसदीय समितिले निरन्तर छलफल चलाउनुको विशेष अर्थ छ । उनी भन्छन्, ‘राज्यका तीनवटै निकाय सीमासम्बन्धी विषयमा चिन्तित छन् भन्ने सन्देश अन्तर्राष्ट्रिय स्तरमा गएको छ, सरकारले यसको सदुपयोग गर्न सक्नुपर्छ ।’ सरकारका प्रवक्ता सूचना तथा सञ्चार प्रविधि मन्त्री गोकुलप्रसाद बास्कोटाले अर्काको एक इन्च जमीन पनि चाहिँदैन र हाम्रो एक इञ्च पनि छाड्दैनौं भनेका थिए । सीमा बचाउ अभियानमा समेत सक्रिय श्रेष्ठ एक इञ्च पनि जमीन छाड्दैनौं भन्ने भनाइ सरकारले व्यवहारिक रुपमै कार्यान्वयन गरेर देखाउनुपर्ने बताउँछन् । सत्तारुढ दल नेपाल कम्युनिस्ट पार्टी नेकपाका सांसदहरु अग्नि सापकोटा, भीम रावललगायतले पनि सरकारलाई तत्काल नयाँ नक्सा सार्वजनिक गर्न सार्वजनिक रुपमा नै आह्वान गर्दै आएका छन् । संसदको राज्य व्यवस्था तथा सुशासन समितिले गत कात्तिक २५ गते कालापानी, लिपुलेक र लिम्पियाधुरालाई समेटेर नेपालको नयाँ नक्सा तयार गर्न सरकारलाई निर्देशन दिएको थियो । त्यसको एक महिनापछि गत मंसिर २७ गते संसदको अन्तर्राष्ट्रिय सम्बन्ध समितिले पनि लिम्पियाधुरा, लिपुलेक, कालापानी र सुस्ता लगायत अतिक्रमित नेपाली भूमि समावेश गरी नेपालको नयाँ नक्सा अविलम्ब जारी गर्न सरकारलाई निर्देशन दिएको छ । दुबै समितिले गृहमन्त्री रामबहादुर थापा र भूमि व्यवस्था मन्त्री पद्मा अर्यालसँगको छलफलपछि यस्तो निर्देशन जारी गरेका हुन् । यसअघि, ७ भदौ २०७६ मा पनि अन्तर्राष्ट्रिय सम्बन्ध समितिले सीमा समस्या समाधानको पहल लिन सरकारलाई निर्देश गरेको थियो । सदन सुरु भएपछि यो विषय जोडतोडले उठ्छ : कांग्रेस प्रमुख प्रतिपक्षी नेपाली कांग्रेसले संसद सुरु भएपछि यो विषयलाई जोडतोडले उठाउने बताएको छ । संसद अधिवेशन पुस ४ गतेका लागि आह्वान भएको छ । ‘जतिवटा संसदीय विषयत समितिले जतिपटक निर्देशन दिएको भए पनि सीमा समस्या समाधानका लागि मन्त्रिपरिषदबाट ठोस निर्णय भएको जानकारीमा आएको छैन । यस विषयमा सदन सुरु भएपछि जोडतोडले कुरा उठाउँछौं’ कांग्रेसकी सचेतक पुष्पा भुसालले अनलाइनखबरसँग भनिन् । प्रधानमन्त्री केपी शर्मा ओलीका परराष्ट्र सल्लाहाकार राजन भट्टराई मिनी संसदको निर्देशन पालनामा सरकार प्रतिवद्ध रहेको बताउँछन् । उचित समयमा नयाँ नक्सा जारी गर्ने सरकारको योजना रहेको उनले बताए । ‘अर्को पक्षसँग वार्ता गर्ने, नयाँ नक्सा जारी गर्ने विषयमा सरकारले पहल गरिरहेको छ । यो बेला राष्ट्रहितको विषयमा संसदीय समितिले चासो दिने, निर्देशनहरु जारी गर्दा ती सकारात्मक हुन्छन्’ भट्टराईले भने । तर, संसदीय समितिहरुले निर्देशन दिँदा वा सांसदहरूले समितिमा बोल्दा आफ्नो मर्यादा र राष्ट्र हितलाई ध्यानमा नराखेको उनको आरोप छ । ‘समितिले निर्देशन दिइरहँदा, र समितिमा बोलिरहँदा सांसदहरुले आफ्नो मर्यादा र राष्ट्रको हितलाई ध्यानमा राख्नुपर्छ भन्ने हो’ भट्टराईले भने । भूमि व्यवस्था मन्त्री पद्माकुमारी अर्यालले पनि सरकारले एउटा सन्दर्भ पारेर नयाँ नक्सा सार्वजनिक गर्ने सार्वजनिक अभिव्यक्ति दिइसकेकी छन् । सीमा समस्या समाधानमा हालसम्मको प्रगति भूमि व्यवस्था मन्त्री अर्यालका अनुसार भारतले नयाँ राजनीतिक नक्सा जारी गरेपछि त्यसको विरोधमा नेपाल सरकारले मुख्य चार वटा काम गरेको छ । ती चार काम हुन्– ‘विज्ञप्तिमार्फत विरोध, राष्ट्रिय सहमति निर्माण, सीमा निरीक्षण समिति गठन र कूटनीतिक नोट पठाउने कार्य । सीमा विवाद सुगौली सन्धिदेखिकै भएको र बेला–बेला सीमा समस्या उठ्ने गरेको भए पनि इतिहासमा कुनै पनि सरकारले हाम्रो सीमा यो हो भनेर भारतीय नक्साको विरोध गरेको थिएन । पछिल्लो समयमा भारतले कालापानी र लिपुलेकलाई समेटेर नयाँ नक्सा जारी गरेपछि वर्तमान सरकारले इतिहासमै पहिलो पटक विज्ञप्ति निकालेर त्यसको विरोध गरेको मन्त्री अर्याल बताउँछिन् । प्रधानमन्त्री केपी शर्मा ओलीले सर्वदलीय, सर्वपक्षीय भेला गराएर सीमा समस्याको विषयमा राष्ट्रिय सहमति कायम गराए । सबैले सरकारलाई सीमा समस्या समाधानका लागि पहल गर्न आग्रह गर्दै सहयोगको प्रतिवद्धता जनाए । यही प्रसंगलाई जोडेर कतिपयले राष्ट्रिय म्यान्डेटअनुसार सरकारले आवश्यक पहल लिन नसक्दा भारतीयहरुले नेपालमा आन्तरिक खपतका लागि यो विषय उठाएको भन्दै टिप्पणी गर्न थालेका छन् । सरकारले नेपाल–भारत र नेपाल चीन सीमा निरीक्षण समिति गठन गर्ने निर्णय गरेको छ । तर, निर्णय भएको एक महिनाभन्दा बढी समय भइसक्दा पनि समिति गठनको प्रक्रिया कहाँ पुग्यो, सार्वजनिक जानकारीमा छैन । मन्त्री अर्यालका अनुसार सीमा निरीक्षण समिति गठनका लागि मन्त्रालयहरुबाट समितिमा सदस्यहरु को को रहने ? भनेर सिफारिस गर्ने कार्य अगाडि बढेको छ । यसैगरी, सीमा विवादको विषयमा नेपाल सरकारले भारत सरकारलाई कूटनीतिक नोट पठाएको छ । यो सबैभन्दा ठूलो कदमका रुपमा सरकारका प्रवक्तासमेत रहेका सञ्चार तथा सूचना प्रविधिमन्त्री गोकुल बास्कोटा अर्थ्याउँछन् । ‘यसलाई नेपालको फस्ट स्टेप मान्नुपर्छ । भारतीय पक्षलाई सत्यका आधारमा सत्यता प्रमाणित गर्न दिनुपर्छ’ गत मंसिर २६ गते मन्त्रालयमै आयोजित पत्रकार सम्मेलनमा मन्त्री बास्कोटाले भनेका थिए । विशेष दूत पठाउने तयारी कालापानीलगायतका क्षेत्रमा देखिएको सीमा विवाद समाधानका लागि परराष्ट्रय सचिव स्तरीय समिति छ । तर समितिको बैठक बस्न सकेको छैन । सर्वे अफ इन्डियाले जारी गरेको भारतको राजनीतिक नक्सामा लिम्पियाधुरा, कालापानी, लिपुलेक लगायतका नेपाली भूमिलाई भारततिर देखाइएको छ । जसको विरोध भएपछि सरकारले कूटनीतिक नोट पठाएको छ । तर, भारतले कूटनीतिक नोटको जवाफ नै नदिएको परराष्ट्र मन्त्रालयका एक उच्च अधिकारीले जानकारी दिए । कूटनीतिक नोटको जवाफ नआइरहेका बेला प्रधानमन्त्री केपी शर्मा ओलीले सीमा समस्या लगायतका विषयमा छलफल गर्न पूर्वप्रधानमन्त्री एवं नेकपा बरिष्ठ नेता माधवकुमार नेपाललाई विशेष दूतका रुपमा भारत पठाउन लागेको विषय चर्चामा अयो । तर, यसको स्वतन्त्र पुष्टि सरकारका प्रतिनिधिले गरेका छैनन् । प्रधानमन्त्रीका परराष्ट्र सल्लाहकार राजन भट्टराई भन्छन्, ‘विशेष दूत भनेर पठाइसकेपछि सबैको जानकारीमा आइहाल्छ । अहिलेलाई यस विषयमा सार्वजनिक रुपमा बोल्ने गरी केही भएको छैन ।
"""

In [22]:
summarizer = Summarize()
summary_news = summarizer.show_summary(
    text=news
    # length_sentence_predict=0.2
    )  # 20% of original length as summary

Loading Embedding
Embedding is now loaded


In [7]:
print(f"Original news: {news}")
print(f"Length of original news: {len(news)}")

Original news: 
१ पुस, काठमाडौं । नापी विभागले लिम्पियाधुरालाई समेटेर नयाँ नक्सा तयार पार्न थालेको छ । सरकारको निर्देशनबमोजिम भारतबाट अतिक्रमित नेपाली भूभागलाई समेटेर नयाँ नक्सा तयार गर्ने प्रक्रिया अगाडि बढेको विभागले जनाएको छ । ‘नयाँ नक्सा तयार गर्ने गरी प्रक्रिया अगाडि बढेको छ’, विभागका सूचना अधिकारी दामोदर ढकालले अनलाइनखबरसँग भने । संघीय संसदका राज्य व्यवस्था समिति र अन्तर्राष्ट्रिय सम्बन्ध समितिले कालापानी, लिपुलेक, लिम्पियाधुरा र सुस्तालगायतका अतिक्रमित नेपाली भूभाग समेटेर नयाँ नक्सा सार्वजनिक गर्न सरकारलाई निर्देशन दिएका छन् । सन् १८१६ मा भएको सुगौली सन्धिअनुसार नेपालको पश्चिमी सिमाना काली (महाकाली) नदी हो । सन्धिअनुसार लिम्पियाधुराबाट बग्ने काली नदी पूर्वका सम्पूर्ण भूभाग नेपालको हो । ती सम्पूर्ण भूभाग समेटेर नक्सा सार्वजनिक गर्न दिएको निर्देशन पालना नभएको भनेर आलोचना भइरहेका बेला विभागले सरकारको निर्देशनमा नयाँ नक्सा बनाउन लागेको हो । विभागका सूचना अधिकारी ढकाल भन्छन्, ‘नापी विभाग सरकारको निर्देशन पालना गर्ने निकाय हो, निर्देशन आएको छ । बाँकी निर्णय उचित समय आएपछि हुन्छ ।’ यो 

In [23]:
print(f"Summary news: {summary_news.strip()}")
print(f"Length of summary news: {len(summary_news)}")

Summary news: सन् १८१६ सुगौली सन्धिअनुसार नेपालको पश्चिमी सिमाना काली महाकाली नदी। यसैगरी सीमा विवादको विषयमा नेपाल सरकारले भारत सरकारलाई कूटनीतिक नोट पठाएको। यसलाई नेपालको फस्ट स्टेप मान्नुपर्छ। सर्वे अफ इन्डियाले जारी भारतको राजनीतिक नक्सामा लिम्पियाधुरा कालापानी लिपुलेक लगायतका नेपाली भूमिलाई भारततिर देखाइएको।
Length of summary news: 300


In [9]:
news2 = """
दाङ — टिपरको ठक्करबाट सोमबार साँझ घोराहीमा एक मोटरसाइकल चालकको मृत्यु भएको छ ।



घोराहीबाट तुलसीपुरतर्फ जाँदै गरेको रा१ख १२९१ नम्बरको टिपरले सोही दिशातर्फ जाँदै गरेको रा७प ७४६ नम्बरको मोटरसाइकललाई घोराही उपमहानगरपालिका–१० झारबैरा चोकमा ठक्कर दिंदा चालक तुलसीपुर उपमहानगरपालिका–१९ कुटीचौरका १९ वर्षीय सुदीप बुढाथोकीको मृत्यु भएको हो ।


दुर्घटनामा घाइते बुढाथोकीको उपचारका क्रममा राप्ती स्वास्थ्य विज्ञान प्रतिष्ठान घोराहीमा मृत्यु भएको जिल्ला प्रहरी प्रवक्ता डीएसपी ईश्वर थापाले बताए । मोटरसाइकलमा सवार कुटीचौरकै २२ वर्षीय दीपक बस्नेत घाइते भएका छन् । उनको राप्ती स्वास्थ्य विज्ञान प्रतिष्ठान घोराहीमा उपचार भइरहेको उनले जानकारी दिए । टिपर चालक घोराही–११, डोक्रेनाका ४० वर्षीय जितबहादुर चौधरीलाई नियन्त्रणमा राखी थप अनुसन्धान गरिरहेको डीएसपी थापाले बताए ।
"""

In [10]:
summary_news2 = summarizer.show_summary(
    text=news2, 
    # length_sentence_predict=0.9
    ) 

In [11]:
print(f"Original news: {news2}")
print(f"Length of original news: {len(news2)}")

Original news: 
दाङ — टिपरको ठक्करबाट सोमबार साँझ घोराहीमा एक मोटरसाइकल चालकको मृत्यु भएको छ ।



घोराहीबाट तुलसीपुरतर्फ जाँदै गरेको रा१ख १२९१ नम्बरको टिपरले सोही दिशातर्फ जाँदै गरेको रा७प ७४६ नम्बरको मोटरसाइकललाई घोराही उपमहानगरपालिका–१० झारबैरा चोकमा ठक्कर दिंदा चालक तुलसीपुर उपमहानगरपालिका–१९ कुटीचौरका १९ वर्षीय सुदीप बुढाथोकीको मृत्यु भएको हो ।


दुर्घटनामा घाइते बुढाथोकीको उपचारका क्रममा राप्ती स्वास्थ्य विज्ञान प्रतिष्ठान घोराहीमा मृत्यु भएको जिल्ला प्रहरी प्रवक्ता डीएसपी ईश्वर थापाले बताए । मोटरसाइकलमा सवार कुटीचौरकै २२ वर्षीय दीपक बस्नेत घाइते भएका छन् । उनको राप्ती स्वास्थ्य विज्ञान प्रतिष्ठान घोराहीमा उपचार भइरहेको उनले जानकारी दिए । टिपर चालक घोराही–११, डोक्रेनाका ४० वर्षीय जितबहादुर चौधरीलाई नियन्त्रणमा राखी थप अनुसन्धान गरिरहेको डीएसपी थापाले बताए ।

Length of original news: 758


In [12]:
print(f"Summary news: {summary_news2.strip()}")
print(f"Length of summary news: {len(summary_news2)}")

Summary news: दाङ टिपरको ठक्करबाट सोमबार साँझ घोराहीमा मोटरसाइकल चालकको मृत्यु। टिपर चालक घोराही११ डोक्रेनाका ४० वर्षीय जितबहादुर चौधरीलाई नियन्त्रणमा राखी थप अनुसन्धान गरिरहेको डीएसपी थापाले बताए।
Length of summary news: 183
